In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from tqdm import tqdm
from torch.nn import LeakyReLU
leaky = LeakyReLU(0.2)
from torch import nn
import torch
import torch.nn.functional as F

# In[3]:


def calculate_score(x, users, aspects):
    niubi = []
    first = x.index[0]
    val_base = [x[first]]
    actor_base = torch.LongTensor(val_base)
    actors_base = aspects(actor_base)
    pre_rating = torch.mm(actors_base, users[first].unsqueeze(1))
    niubi.append(pre_rating)
    pre_rating = pre_rating / pre_rating.shape[0]
    pre_rating = pre_rating.sum(0)
    for i in x.index[1:]:
        val = [x[i]]
        actor = torch.LongTensor(val)
        actors = aspects(actor)
        pre_ra = torch.mm(actors, users[i].unsqueeze(1))
        niubi.append(pre_ra)
        actors_f = pre_ra / pre_ra.shape[0]
        actors_f = actors_f.sum(0)
        pre_rating = torch.cat((pre_rating, actors_f))
    return pre_rating, niubi  ### pre_rating is the rating after averaging on the type; niu is the rating before averaging on the type


class aspect_augumentation_music(nn.Module):
    def __init__(self, n_users, n_entity, n_rk, n_factors):
        super(aspect_augumentation_music, self).__init__()
        
        
        '''
        n_users denotes the numeber of users
        n_entity denotes the number of attributes
        n_rk denotes the number of attribute type
        n_factors denotes the embedding size
        '''
        self.n_users = n_users
        self.n_entity = n_entity
        self.n_rk = n_rk
        self.n_factors = n_factors
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.entity_factors = torch.nn.Embedding(n_entity, n_factors)
        self.relation_k = torch.nn.Embedding(n_factors, n_rk)

    def forward(self, user_id, artists_id, categories_id,rate):
        '''
        user_factors: n_users * 64
        entity_factors: n_entity * 64
        relation_k: 64 * 3
        artists_id is the index of authors
        categories_id is the the index of genres
        '''

        users = self.user_factors(user_id)  # 128 * 8 We get the representation of users
        aspects = self.entity_factors  # n_entity * 8 We get the representation of aspects
        
        
        users = F.dropout(users, p=rate, training=self.training) ## Here we apply dropout 
        
        '''
        Compute the importance of each aspects
        '''
        scores = torch.matmul(users, F.dropout(
            self.relation_k.weight, p=rate, training=self.training))  # 128 * 3 Here, we compute the score of each attribute type to users, Equation 1 in the paper
        scores = leaky(scores)
        m = torch.nn.Softmax(dim=1)  # 128 * 3 
        scores = m(scores)  # 128 * 3 We normalize the score to get the importance of each attribute type to users, Equation 2 in the paper


        '''
        Compute the importance of each aspects
        '''
        scores_actors = scores[:, 0]  # 128,
        scores_directors = scores[:, 1]  # 128,

        '''
        Compute the contribution of each aspects
        '''
        contribute_actors, niubi_act = calculate_score(
            artists_id, users, aspects)
        contribute_directors, niubi_dir = calculate_score(
            categories_id, users, aspects)

        '''
        Compute the final predictions
        '''
        importance_sum = scores_actors + scores_directors
        prediction_sum = contribute_actors * scores_actors + \
            contribute_directors * scores_directors
        prediction = prediction_sum / importance_sum
        cnm = [niubi_act, niubi_dir]

        return prediction, scores, contribute_actors, contribute_directors, cnm
    
        '''
        prediction is the the prediction of ratings; 
        scores represent the importance of each attribute type;
        contribute_actors denote the contribution of authors;
        contribute_directors denote the contribution of genres;
        cnm is score before averaging
        ''' 


# In[ ]:






In [11]:
from data_loader import read_attributes

In [13]:
a = read_attributes('music')

In [16]:
print(a)

None


In [34]:
ratings = pd.read_excel('data/music/Data_InCarMusic.xlsx', usecols=[0,1,2], sheet_name='ContextualRating')
       
    
music_df = pd.read_excel('data/music/Data_InCarMusic.xlsx', usecols=[0,2,7], sheet_name='Music Track')
genre_map = {1: 'Blues music', 2: 'Classical music', 3: 'Country music', 4: 'Disco music', 5: 'Hip Hop music', 6: 'Jazz music',
              7: 'Metal music', 8: 'Pop music', 9: 'Reggae music', 10: 'Rock music'}
music_df = music_df.rename(columns = {' category_id':'category_id',' artist':'artist'})
music_df['category_id'] = music_df['category_id'].map(genre_map)


In [35]:
ratings.columns

Index(['UserID', 'ItemID', ' Rating'], dtype='object')

In [36]:
music_df.columns

Index(['id', 'artist', 'category_id'], dtype='object')

In [37]:


  # Stock the attributes of music in a dictionary
info = {}
for ind in music_df.index:
    music = music_df['id'][ind]
    info.setdefault(music, {})
    info[music]['artist'] = [music_df['artist'][ind]]
    info[music]['category'] = [music_df['category_id'][ind]]

music_category = dict_movie_aspect(info, "category")
music_artist = dict_movie_aspect(info, 'artist')

cartegory = pd.DataFrame.from_dict(
        music_category, dtype='int64', orient='index').T.replace(np.nan, 0).index
artist = pd.DataFrame.from_dict(
        music_artist, dtype='int64', orient='index').T.replace(np.nan, 0).index

category_dict = dict(zip(cartegory, range(len(cartegory))))
artist_dict = dict(
        zip(artist, range(len(cartegory), len(cartegory) + len(artist))))

<ipython-input-37-84ab6652ac08>:12: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  cartegory = pd.DataFrame.from_dict(
<ipython-input-37-84ab6652ac08>:14: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  artist = pd.DataFrame.from_dict(


In [39]:
music_df.columns

Index(['id', 'artist', 'category_id'], dtype='object')

In [40]:
ratings = pd.read_excel('data/music/Data_InCarMusic.xlsx', usecols=[0,1,2], sheet_name='ContextualRating')
       
    
music_df = pd.read_excel('data/music/Data_InCarMusic.xlsx', usecols=[0,2,7], sheet_name='Music Track')

In [43]:
ratings.columns

Index(['UserID', 'ItemID', ' Rating'], dtype='object')

In [42]:
music_df

,id,artist,category_id
0,248,B.B.King,1
1,250,Mamie Smith,1
2,251,Robert Johnson,1
3,252,T-Bone Walker,1
4,253,Antonio Vivaldi,2
...,...,...,...
134,759,Fettes Brot,8
135,761,Lady GaGa,8
136,760,Bon Jovi,10
137,762,DJ Gollum vs. Basslovers United,4
